In [31]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [32]:
house_dict = []

for page in range(17, 21):
    linked = 'https://www.zillow.com/ca/townhomes/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-153.847663625,"east":-84.765632375,"south":12.777434491046995,"north":56.013075839104054},"mapZoom":4,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"isPreMarketPreForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isLotLand":{"value":false},"doz":{"value":"90"},"isApartment":{"value":false},"isPreMarketForeclosure":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
            print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
            
            
            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[0].text
                if (zestimate == "Zestimate®"):
                    zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[1].text
            except:
                zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = 'null'
#             print(zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(2)
            
        except AttributeError as e:
            continue

print(house_dict)

$670,659+
$652,902+
$749,990+
$699,825+
$532,900+
$414,575+
$549,900+
$418,880+
$624,380+
$446,880+
$761,990+
$509,990+
$588,000+
$616,990+
$1,150,000+
$899,990+
$1,269,000+
$879,990+
$946,202+
$1,489,187+
$757,372+
$1,140,000+
$471,000+
$635,000+
$390,000
$375,880+
$379,990+
$899,990+
$1,090,995+
$914,990+
$688,000
$474,990+
$831,880+
$744,990+
$723,990+
$1,869,880+
$425,000
$1,588,000
$820,000+
$899,000
$448,999
$1,049,800
$509,000
$339,900
$812,990+
$505,880+
$795,000+
$699,999
$880,000+
$673,235+
$595,000
$1,188,880+
$884,990+
$890,348
$619,050
$646,500
$727,755
$771,900+
$820,000+
$469,000
$775,000
$799,500
$4,275,000
$1,299,999
$818,000+
$1,245,000
$492,900+
$770,005
$778,270
$786,985
$512,861
$585,000
$644,990+
$920,880+
$509,874+
$721,900+
$497,900+
$969,990+
$909,990+
$595,880+
$595,880+
$479,900+
$831,990+
$482,380+
$479,990+
$484,990+
$475,000+
$775,450+
$675,450+
$831,850+
$1,518,995+
$1,554,095+
$764,990+
$1,316,880+
$1,073,880+
$1,075,000+
$491,000+
$707,689+
$675,000+
$6

In [33]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,HOA,House_type,House_website,Listing_type,Lot,Lot_conversion_to_sqft,Parking,Price,Rent_zestimate,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,3,2,OakWalk,"The Griffin - Plan 1A Plan, Oak Walk",$283/month,Townhouse,https://www.zillow.com/community/oak-walk/2082...,New construction,No Data,null,2 spaces,"$670,659+",null,"1,326",null,The Griffin - Plan 1A Plan,2020,null,$506,null
1,3,3,VintageatOldTownTustin,"Hudson Plan 1 Plan, Vintage at Old Town Tustin",null,Townhouse,https://www.zillow.com/community/vintage-at-ol...,New construction,No Data,null,2 spaces,"$652,902+",null,"1,703",null,Hudson Plan 1 Plan,2020,null,$383,null
2,4,3,CandlewoodVillas,"3 Plan, Candlewood Villas",null,Townhouse,https://www.zillow.com/community/candlewood-vi...,New construction,No Data,null,2 spaces,"$749,990+",null,"1,789",null,3 Plan,2020,null,$419,null
3,3,3,Lewis+Mason,"Mason 1 Plan, Lewis+Mason",null,Townhouse,https://www.zillow.com/community/lewis-mason/2...,New construction,No Data,null,2 spaces,"$699,825+",null,"1,858",null,Mason 1 Plan,2020,null,$377,null
4,4,4,Highline,"Plan 1B Plan, Highline",$308/month,Townhouse,https://www.zillow.com/community/highline/2082...,New construction,No Data,null,2 spaces,"$532,900+",null,"2,026",null,Plan 1B Plan,2020,null,$263,null
5,3,2,SolsticeatDayCreekSquare,"Residence 1295 Plan, Solstice at Day Creek Square",null,Townhouse,https://www.zillow.com/community/solstice-at-d...,New construction,No Data,null,2 spaces,"$414,575+",null,"1,295",null,Residence 1295 Plan,2020,null,$320,null
6,4,2,Horizons,"2 Plan, Horizons",null,Townhouse,https://www.zillow.com/community/horizons/2081...,New construction,No Data,null,2 spaces,"$549,900+",null,"1,858",null,2 Plan,2020,null,$296,null
7,2,3,IndigoatCanvasParkatNewHaven,"Residence 3X Plan, Indigo at Canvas Park at Ne...",null,Townhouse,https://www.zillow.com/community/indigo-at-can...,New construction,No Data,null,2 spaces,"$418,880+",null,"1,517",null,Residence 3X Plan,2020,null,$276,null
8,4,3,PalazzoatDayCreekSquare,"Residence 2183 Plan, Palazzo at Day Creek Square",null,Townhouse,https://www.zillow.com/community/palazzo-at-da...,New construction,No Data,null,2 spaces,"$624,380+",null,"2,183",null,Residence 2183 Plan,2020,null,$286,null
9,3,2,VerandaatDayCreekSquare,"Residence 1561 Plan, Veranda at Day Creek Square",null,Townhouse,https://www.zillow.com/community/veranda-at-da...,New construction,No Data,null,2 spaces,"$446,880+",null,"1,561",null,Residence 1561 Plan,2020,null,$286,null


In [34]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2020_Townhouse_Data_page17-20.csv", index=False, header=True)